# Construcción del modelo en mediapipe

In [ ]:
# Verificación que se este utilizando la GPU
import torch
print("¿CUDA disponible?", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0))

¿CUDA disponible? True
GPU actual: NVIDIA GeForce RTX 4070 Ti



### 🔹 Explicación de Importaciones

#### **Librerías estándar y manejo de archivos**

* **`os`**

  * Utilizada para interactuar con el sistema operativo (manejar rutas, listar directorios y archivos).
  * Ejemplo de uso: `os.listdir()`, `os.path.join()`.

#### **Procesamiento de Imágenes y Videos**

* **`cv2`** (`opencv-python`)

  * Se utiliza para la lectura, escritura y redimensionamiento de imágenes extraídas de los videos.
  * Ejemplo de uso: `cv2.imread()`, `cv2.resize()`.

#### **Cómputo numérico y manejo de datos**

* **`numpy as np`**

  * Se emplea para manejar arrays multidimensionales, especialmente para cargar vectores guardados en formato `.npy`.
  * Ejemplo de uso: `np.load()`, `np.array()`.

#### **Herramientas de Machine Learning y Deep Learning (PyTorch)**

* **`torch`**

  * Framework central para la creación y entrenamiento del modelo.
  * Ejemplo de uso: Tensores (`torch.tensor()`), cálculo de gradientes (`loss.backward()`).

* **`torch.utils.data`** (`Dataset`, `DataLoader`)

  * `Dataset`: Clase base para definir un conjunto personalizado de datos (manejo de muestras y etiquetas).
  * `DataLoader`: Herramienta para cargar datos en batches durante el entrenamiento.

* **`torchvision.transforms`**

  * Utilizado para realizar transformaciones comunes en imágenes (por ejemplo, conversión de imágenes a tensores).

* **`torchvision.models as models`**

  * Proporciona modelos preentrenados, en este caso EfficientNet B0, que se utiliza como extractor de características visuales.

* **`torch.nn as nn`**

  * Define bloques fundamentales de redes neuronales (capas convolucionales, lineales, funciones de activación, etc.).
  * Ejemplo de uso: `nn.LSTM`, `nn.Linear`, `nn.ReLU`, `nn.Sigmoid`.

* **`torch.optim` (Adam)**

  * Adam es un optimizador eficiente usado para actualizar los parámetros del modelo durante el entrenamiento.

#### **Evaluación del Modelo**

* **`sklearn.metrics` (`accuracy_score`, `f1_score`, `classification_report`)**

  * Métricas utilizadas para evaluar el rendimiento del modelo.
  
#### **Utilidades diversas**

* **`sklearn.utils.shuffle`**

  * Se utiliza para mezclar aleatoriamente los datos antes de dividirlos en entrenamiento y validación.

* **`collections.Counter`**

  * Herramienta para contar ocurrencias de elementos en listas, usada aquí para mostrar distribución de clases y predicciones del modelo.





---
### **Instalación de Dependencias**

A continuación, se presentan los comandos necesarios para instalar todas las librerías usadas en este notebook. Se recomienda hacerlo dentro de un entorno virtual:

```bash
# Librerías básicas para manipulación de imágenes y arrays
pip install opencv-python
pip install numpy

# PyTorch (GPU o CPU según disponibilidad)
# Si tienes CUDA disponible:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Si solo tienes CPU (sin CUDA):
pip install torch torchvision torchaudio

# Librerías de Machine Learning adicionales
pip install scikit-learn

# Utilidad adicional (opcional, generalmente instalada por defecto)
pip install collections
```

---

## 🛠️ **Configuración recomendada de entorno virtual**

Es recomendable crear un entorno virtual para manejar dependencias sin conflictos:

```bash
python -m venv .venv

# Activar entorno virtual:
# Windows
.venv\Scripts\activate

# Linux / MacOS
source .venv/bin/activate
```

Luego ejecuta los comandos de instalación arriba mencionados.

---

**Nota importante:**

* **PyTorch** tiene dos variantes principales según si dispones de una GPU con CUDA o solo CPU. Asegúrate de seleccionar la instalación correspondiente a tu hardware.



In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score, classification_report
from collections import Counter
import torch.nn as nn
from torch.optim import Adam



### **Explicación del Dataset Personalizado**

#### 🔹 Clase `DeepfakeDataset`

Esta clase personalizada hereda de la clase base `torch.utils.data.Dataset` y es utilizada para cargar y gestionar eficientemente el conjunto de datos del proyecto, que consiste en imágenes de rostros y vectores extraídos con MediaPipe. Su principal objetivo es facilitar el manejo estructurado de secuencias de imágenes (frames) y vectores numéricos para entrenamiento y validación del modelo de detección de deepfakes.

---

#### **Métodos Principales**

##### **1. Constructor (`__init__`)**

**Parámetros:**

* **`base_path`**: Ruta raíz que contiene las carpetas de datos ("original" y diversas carpetas de deepfakes).
* **`sequence_length=16`**: Número de imágenes (frames) consecutivas utilizadas para representar cada video.
* **`split='train'`**: Determina si el conjunto se utiliza para entrenamiento (`train`) o validación (`val`).
* **`val_split=0.2`**: Proporción de datos reservados para la validación.

**Funcionamiento interno:**

* Carga todos los videos completos utilizando la función interna `_collect_video_ids()`.
* Mezcla aleatoriamente todos los videos para evitar sesgos en la división.
* Realiza la división del dataset a nivel video, no por imagen, garantizando que imágenes del mismo video nunca estén simultáneamente en entrenamiento y validación.

---

##### **2. Método privado `_collect_video_ids(base_path)`**

Esta función realiza un recorrido estructurado sobre los datos:

**Funcionamiento:**

* Itera sobre cada carpeta en `base_path`.
* Determina la etiqueta (`label`):

  * `0`: si la carpeta es `"original"` (videos reales).
  * `1`: si es cualquier otra carpeta (videos falsificados o deepfake).
* Agrupa imágenes por ID de video (por ejemplo, `video123_0.jpg`, `video123_1.jpg`, etc.).
* Selecciona únicamente los videos que contienen al menos tantas imágenes como la `sequence_length` requerida, descartando videos incompletos.

**Devuelve:**

* Una lista de tuplas con la estructura `(ruta_carpeta, video_id, etiqueta)`.

---

##### **3. Método `__len__(self)`**

* Simplemente devuelve la cantidad total de videos disponibles en el conjunto actual (`train` o `val`).

---

##### **4. Método `__getitem__(self, idx)`**

Este es el método clave, llamado por `DataLoader` para obtener una muestra individual del conjunto de datos:

**Funcionamiento:**

* Recupera el video correspondiente al índice dado (`idx`).
* Para cada video, carga exactamente `sequence_length` imágenes (`.jpg`) y sus correspondientes vectores de landmarks faciales (`.npy`).
* Cada imagen se carga usando OpenCV (`cv2.imread()`), redimensionada a `256x256`, y transformada en un tensor mediante `torchvision.transforms`.
* Los landmarks (vectores numéricos) se cargan directamente desde archivos `.npy`.
* Finalmente, apila todas las imágenes y landmarks en tensores con dimensiones:

  * Imágenes: `(sequence_length, 3, 256, 256)`
  * Landmarks: `(sequence_length, 5)`
* Devuelve una tupla: `(tensor_imágenes, tensor_landmarks, etiqueta)`

---

### **Función de diagnóstico `print_class_distribution(dataset, name)`**

Función utilitaria adicional, utilizada para verificar la distribución de clases (real vs fake) en un conjunto de datos dado:

**Funcionamiento:**

* Recorre el dataset proporcionado, acumulando las etiquetas.
* Utiliza la clase `Counter` para contar cuántas muestras hay de cada clase (`0`: real, `1`: fake).
* Muestra claramente en pantalla la distribución de clases, útil para verificar el balance del dataset antes del entrenamiento.

**Ejemplo de salida:**

```
Train class distribution:
  Real (0): 1200
  Fake (1): 800
------------------------------
```

---

### **Resumen de uso:**

Al utilizar la clase `DeepfakeDataset`, se asegura:

* **Consistencia en las muestras:** Cada muestra devuelta contiene siempre secuencias completas, facilitando entrenamiento y evaluación del modelo.
* **Separación correcta:** Los videos de entrenamiento y validación nunca se mezclan, evitando la fuga de datos (data leakage).
* **Control de calidad:** La función de diagnóstico permite rápidamente verificar que las clases estén equilibradas y listas para el entrenamiento.



In [ ]:
class DeepfakeDataset(Dataset):
    def __init__(self, base_path, sequence_length=16, split='train', val_split=0.2):
        self.sequence_length = sequence_length
        self.base_path = base_path
        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        # Paso 1: Cargar videos completos con etiquetas
        all_videos = self._collect_video_ids(base_path)
        all_videos = shuffle(all_videos, random_state=42)  # Mezcla antes de dividir
        
        # Paso 2: Dividir por video, no por muestra
        split_idx = int(len(all_videos) * (1 - val_split))
        self.videos = all_videos[:split_idx] if split == 'train' else all_videos[split_idx:]

    def _collect_video_ids(self, base_path):
        video_samples = []
        for folder in os.listdir(base_path):
            folder_path = os.path.join(base_path, folder)
            label = 0 if folder == "original" else 1
            video_ids = {}
            for file in os.listdir(folder_path):
                if file.endswith(".jpg"):
                    vid = "_".join(file.split("_")[:-1])
                    video_ids.setdefault(vid, []).append(file)
            for vid, files in video_ids.items():
                if len(files) >= self.sequence_length:
                    video_samples.append((folder_path, vid, label))
        return video_samples

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        folder, video_id, label = self.videos[idx]
        imgs = []
        lmks = []
        for i in range(self.sequence_length):
            img_path = os.path.join(folder, f"{video_id}_{i}.jpg")
            lmk_path = os.path.join(folder, f"{video_id}_{i}.npy")
            img = cv2.imread(img_path)
            img = cv2.resize(img, (256, 256))
            img = self.transform(img)  # (3, 256, 256)
            lmk = torch.tensor(np.load(lmk_path), dtype=torch.float32)
            imgs.append(img)
            lmks.append(lmk)
        imgs = torch.stack(imgs)      # (16, 3, 256, 256)
        lmks = torch.stack(lmks)      # (16, 5)
        return imgs, lmks, torch.tensor(label, dtype=torch.float32)
    

# ✅ Diagnóstico: Conteo de clases reales/falsas
def print_class_distribution(dataset, name):
    labels = [int(label.item()) for _, _, label in dataset]
    counts = Counter(labels)
    print(f"{name} class distribution:")
    print(f"  Real (0): {counts.get(0, 0)}")
    print(f"  Fake (1): {counts.get(1, 0)}")
    print("-" * 30)



### **Explicación del Modelo DeepfakeDetector**

La clase `DeepfakeDetector` implementa una red neuronal híbrida que combina extracción de características mediante redes convolucionales preentrenadas y modelado secuencial mediante redes recurrentes (LSTM). El modelo está diseñado específicamente para detectar videos falsificados (deepfakes) analizando secuencias de imágenes junto con vectores numéricos asociados a landmarks faciales.

---

### 🔹 **Arquitectura del Modelo**

La arquitectura está compuesta principalmente por tres partes:

1. **CNN (EfficientNet-B0)**
2. **LSTM bidireccional**
3. **Capa de clasificación (Fully Connected)**

---

#### **1. Red Convolucional CNN (EfficientNet-B0)**

```python
self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
self.cnn.classifier = nn.Identity()  # elimina la capa final
```

**Descripción:**

* Utiliza el modelo EfficientNet-B0 preentrenado en ImageNet, que es altamente eficiente en términos de precisión y tamaño.
* Se remueve la capa clasificadora original (última capa fully-connected) mediante `nn.Identity()`. Esto permite usar EfficientNet como extractor de características visuales (embedding visual de dimensión 1280).

**Resultado:**
Cada imagen pasa por EfficientNet-B0, generando un vector de características de tamaño **1280**.

---

#### **2. Red Recurrente LSTM Bidireccional**

```python
self.lstm = nn.LSTM(
    input_size=1285,  # embedding visual (1280) + landmarks (5)
    hidden_size=128,
    num_layers=1,
    batch_first=True,
    bidirectional=True
)
```

**Descripción:**

* La red LSTM es capaz de capturar dependencias temporales entre frames consecutivos del video.
* El `input_size` de 1285 corresponde a la combinación de:

  * **1280 dimensiones** del embedding visual generado por EfficientNet.
  * **5 dimensiones** del vector de landmarks faciales obtenidos con MediaPipe.
* Es bidireccional, permitiendo modelar información temporal en ambas direcciones (adelante y atrás en la secuencia).
* El resultado es un vector combinado de tamaño `256` (128 dimensiones × 2 direcciones).

**Resultado:**
Captura información dinámica y temporal para distinguir patrones entre videos reales y falsificados.

---

#### **3. Clasificación mediante capa Fully-Connected**

```python
self.fc = nn.Sequential(
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 1),
    nn.Sigmoid()
)
```

**Descripción:**

* Esta capa recibe el vector resultante del LSTM y produce una predicción binaria (real o falso).
* Contiene:

  * Una capa lineal que reduce dimensionalidad de 256 → 64.
  * Activación ReLU para introducir no-linealidad.
  * Dropout de `0.3` para reducir sobreajuste (overfitting).
  * Una capa lineal final que reduce la dimensionalidad de 64 → 1.
  * Activación Sigmoid para convertir la salida en una probabilidad entre 0 y 1 (clasificación binaria).

**Resultado:**
Una probabilidad indicando si el video es real (`0`) o falsificado (`1`).

---

### **Método `forward(x_imgs, x_lmks)`**

El método `forward` define cómo fluye la información a través del modelo durante la inferencia o entrenamiento:

**Parámetros:**

* **`x_imgs`**: Tensor con dimensiones `(Batch, Tiempo, Canales, Alto, Ancho)` que contiene las secuencias de imágenes.
* **`x_lmks`**: Tensor con dimensiones `(Batch, Tiempo, 5)` que contiene secuencias de vectores de landmarks faciales.

**Proceso:**

1. **Preprocesamiento de Imágenes:**

   * Se reordenan imágenes en un único lote (`B*T`) para pasar todas por EfficientNet simultáneamente.

2. **Extracción de características visuales:**

   * EfficientNet procesa las imágenes para obtener embeddings visuales (`1280` dimensiones por imagen).

3. **Combinar embeddings visuales y landmarks:**

   * Las características visuales (`1280`) y los landmarks (`5`) se concatenan, formando un tensor combinado (`1285` dimensiones por frame).

4. **Procesamiento secuencial con LSTM:**

   * El tensor combinado pasa por la red LSTM bidireccional, que captura información temporal.

5. **Clasificación final:**

   * Se utiliza solamente la última salida de la secuencia LSTM (último frame procesado).
   * Este vector final se introduce en la capa de clasificación fully-connected.
   * Se obtiene finalmente una predicción en forma de probabilidad.

**Dimensiones de Tensores:**

* Entrada: Imágenes `(B, 16, 3, 256, 256)` y Landmarks `(B, 16, 5)`
* CNN: `(B*16, 1280)`
* Después CNN: `(B, 16, 1280)`
* Combinado con landmarks: `(B, 16, 1285)`
* Después LSTM: `(B, 16, 256)`
* Salida final: `(B,)` (probabilidad binaria)

---

### **Resumen de uso:**

El modelo es un detector especializado en analizar tanto patrones visuales como temporales en videos para identificar deepfakes, combinando técnicas avanzadas de visión computacional y modelado secuencial.




In [ ]:
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.cnn.classifier = nn.Identity()  # elimina la capa final
        self.embedding_dim = 1280
        self.sequence_length = 16

        self.lstm = nn.LSTM(input_size=1285, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x_imgs, x_lmks):
        B, T, C, H, W = x_imgs.shape
        x_imgs = x_imgs.view(B * T, C, H, W)
        features = self.cnn(x_imgs)                     # (B*T, 1280)
        features = features.view(B, T, -1)              # (B, T, 1280)
        combined = torch.cat([features, x_lmks], dim=2) # (B, T, 1285)
        out, _ = self.lstm(combined)
        out = out[:, -1, :]                             # última salida
        return self.fc(out).squeeze(1)


In [24]:
base_path = r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe"

train_dataset = DeepfakeDataset(base_path, split='train')
val_dataset = DeepfakeDataset(base_path, split='val')

print_class_distribution(train_dataset, "Train")
print_class_distribution(val_dataset, "Validation")


Train class distribution:
  Real (0): 768
  Fake (1): 723
------------------------------
Validation class distribution:
  Real (0): 207
  Fake (1): 166
------------------------------


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset y loaders
train_dataset = DeepfakeDataset(base_path=r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe", split='train')
val_dataset = DeepfakeDataset(base_path=r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe", split='val')

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Modelo
model = DeepfakeDetector().to(device)
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCELoss()

best_f1 = 0.0

for epoch in range(1, 11):
    model.train()
    running_loss = 0.0
    for imgs, lmks, labels in train_loader:
        imgs, lmks, labels = imgs.to(device), lmks.to(device), labels.to(device)
        preds = model(imgs, lmks)
        loss = criterion(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Validación
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, lmks, labels in val_loader:
            imgs, lmks = imgs.to(device), lmks.to(device)
            outputs = model(imgs, lmks)
            preds = (outputs > 0.5).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Epoch {epoch}: Loss={avg_loss:.4f} | Val Accuracy={acc:.4f} | Val F1-score={f1:.4f}")

    # Guardar mejor modelo
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "mediapipe_model.pth")
        print(f"✅ Modelo guardado con F1-score = {f1:.4f}")


Epoch 1: Loss=0.6595 | Val Accuracy=0.7507 | Val F1-score=0.7424
✅ Modelo guardado con F1-score = 0.7424
Epoch 2: Loss=0.5283 | Val Accuracy=0.8874 | Val F1-score=0.8750
✅ Modelo guardado con F1-score = 0.8750
Epoch 3: Loss=0.3916 | Val Accuracy=0.8740 | Val F1-score=0.8698
Epoch 4: Loss=0.3354 | Val Accuracy=0.9008 | Val F1-score=0.8969
✅ Modelo guardado con F1-score = 0.8969
Epoch 5: Loss=0.2427 | Val Accuracy=0.9303 | Val F1-score=0.9217
✅ Modelo guardado con F1-score = 0.9217
Epoch 6: Loss=0.1966 | Val Accuracy=0.7909 | Val F1-score=0.8079
Epoch 7: Loss=0.1488 | Val Accuracy=0.9491 | Val F1-score=0.9408
✅ Modelo guardado con F1-score = 0.9408
Epoch 8: Loss=0.1300 | Val Accuracy=0.9357 | Val F1-score=0.9245
Epoch 9: Loss=0.0682 | Val Accuracy=0.9196 | Val F1-score=0.9143
Epoch 10: Loss=0.0860 | Val Accuracy=0.9357 | Val F1-score=0.9264


In [ ]:
print("Preds:", Counter(np.array(y_pred, dtype=int)))

Preds: Counter({0: 213, 1: 160})


In [ ]:
print(classification_report(y_true, y_pred, target_names=["real", "fake"]))

              precision    recall  f1-score   support

        real       0.93      0.96      0.94       207
        fake       0.94      0.91      0.93       166

    accuracy                           0.94       373
   macro avg       0.94      0.93      0.93       373
weighted avg       0.94      0.94      0.94       373

